# Interpolation datapath generator

### Parameters

In [72]:
import math 
from datetime import datetime

In [73]:
####################################################
# Inputs definitions
####################################################
INPUT_N = 4                                     # Input size is a INPUT_N x INPUT_N block of 4x4 sub-blocks
INPUT_COORD_WIDTH = 8                           # Number of bits as input for each block in an input line
INPUT_MACROBLOCK_SIZE = 16                      # Number of pixels in one macroblock (16 = 4x4)
PIXEL_WIDTH = 8                                 # Number of bits for each integer pixel from the reference frame


####################################################
# Interpolator definitions
####################################################
INPUT_LINE_SIZE = 9                             # Input size for ONE line loaded to the buffer (9 integer pixels)
TAPS = 6                                        # Number of taps
FILTERS = 15                                    # Number of filters (= number of outputs expected)
OUT_1ST_INTERP_WIDTH = 11                       # Width for every 15 interpolation output for FIRST PASS
OUT_2ND_INTERP_WIDTH = 14                       # Width for every 15 interpolation output for SECOND PASS
BUFFER_LINES = 9                                # Number of lines in the transposed buffer
BUFFER_COLUMNS = 4                              # Number of columns in the transposed buffer


####################################################
# Motion Vector Generator definitions
####################################################
INPUT_COORD_WIDTH = 8                           # Input size for the coordinates X and Y
INPUT_MV_WIDTH = 8                              # Input size for the block motion vectors (MV0 and MV1)
OUTPUT_MV_GEN_WIDTH = 19                        # Output size of the Motion Vector module (4 and 6 parameters are equal)


####################################################
# INTERPOLATOR DATAPATH DEFINITIONS
####################################################
PARALLEL = 1                                    # Number of lines to be processed each time the interpolator runs. 
                                                #       Obs: 1 line has 9 integer samples
SAMPLES_PER_LINE = 4                            # Number of samples in one line
INDEX_0 = 2                                     # Which position to be considered to input the buffer
INT_FILE_NAME = 'interpolation_datapath'        # Interpolation datapath file name
INT_DESCRIPTION = 'Interpolation datapath to process '


### Intermediate variables calculation

In [74]:
####################################################
# INTERPOLATOR VARIABLES
####################################################
# Modified date
DATE_MODIF = str(datetime.today().strftime('%d/%m/%Y'))

# Inputs from control and MV Gen datapath [signal name, bit width, io type, description]
# Obs: io type is relative to this datapath pov
INT_CTRL_SIGNALS = [
    ['CLK', 1, 'input', ''],
    ['RST_ASYNC_INTERP', 1, 'input', '\t\t\t// Async reset for regs INPUT_LINE, INT_OUT and BUFFER.'],
    ['RST_ASYNC_REG_COUNTER', 1, 'input', ''],

    ['WRITE_REG_INPUT_LINE', 1, 'input', ''],
    ['WRITE_REG_COUNTER', 1, 'input', ''],
    ['WRITE_REG_INT_OUT', 1, 'input', ''],
    ['WRITE_BUFFER', 1, 'input', ''],

    ['SEL_BUFFER_IN', 1, 'input', '\t\t\t// 0: 1st pass output; 1: OUT_REG_INPUT_LINE'],
    ['SEL_INTERP_IN', 1, 'input', '\t\t\t// 0: buffer; 1: OUT_REG_INPUT_LINE'],
    ['SEL_DIMENSION', 1, 'input', '\t\t\t// 0: Y; 1: X'],

    ['INTEGER_SAMPLES', PIXEL_WIDTH*INPUT_LINE_SIZE, 'input', ''],
    ['FRAC_MV_X',  int(math.log2((FILTERS+1))), 'input', '\t\t\t// Fractional part of MV_X from MV Generator.'],
    ['FRAC_MV_Y',  int(math.log2((FILTERS+1))), 'input', '\t\t\t// Fractional part of MV_Y from MV Generator.'],

    ['LOOP_3', 1, 'output wire', ''],
    ['LOOP_4', 1, 'output wire', ''],
    ['LOOP_9', 1, 'output wire', ''],
    ['LOOP_13', 1, 'output wire', ''],

    ['INTERP_OUT_0', OUT_2ND_INTERP_WIDTH, 'output wire', ''],
    ['INTERP_OUT_1', OUT_2ND_INTERP_WIDTH, 'output wire', ''],
    ['INTERP_OUT_2', OUT_2ND_INTERP_WIDTH, 'output wire', ''],
    ['INTERP_OUT_3', OUT_2ND_INTERP_WIDTH, 'output wire', ''],

    # TESTBENCH SIGNALS
    ['TB_OUT_REG_INPUT_LINE', INPUT_LINE_SIZE*PIXEL_WIDTH, 'output wire', ''],
    ['TB_OUT_MUX_SEL_FILTER_0', OUT_2ND_INTERP_WIDTH, 'output wire', ''],
    ['TB_OUT_MUX_SEL_FILTER_1', OUT_2ND_INTERP_WIDTH, 'output wire', ''],
    ['TB_OUT_MUX_SEL_FILTER_2', OUT_2ND_INTERP_WIDTH, 'output wire', ''],
    ['TB_OUT_MUX_SEL_FILTER_3', OUT_2ND_INTERP_WIDTH, 'output wire', ''],
    ['TB_BUF_COL_0', INPUT_LINE_SIZE*OUT_1ST_INTERP_WIDTH, 'output wire', ''],
    ['TB_BUF_COL_1', INPUT_LINE_SIZE*OUT_1ST_INTERP_WIDTH, 'output wire', ''],
    ['TB_BUF_COL_2', INPUT_LINE_SIZE*OUT_1ST_INTERP_WIDTH, 'output wire', ''],
    ['TB_BUF_COL_3', INPUT_LINE_SIZE*OUT_1ST_INTERP_WIDTH, 'output wire', ''],
    ['TB_OUT_MUX_SEL_INTERP_LINE_IN', INPUT_LINE_SIZE*OUT_1ST_INTERP_WIDTH, 'output wire', ''],
    ['TB_IN_ALL_FILTERS_0', TAPS*OUT_1ST_INTERP_WIDTH, 'output wire', ''],
    ['TB_IN_ALL_FILTERS_1', TAPS*OUT_1ST_INTERP_WIDTH, 'output wire', ''],
    ['TB_IN_ALL_FILTERS_2', TAPS*OUT_1ST_INTERP_WIDTH, 'output wire', ''],
    ['TB_IN_ALL_FILTERS_3', TAPS*OUT_1ST_INTERP_WIDTH, 'output wire', ''],
    ['TB_OUT_ADD_COUNTER', 4, 'output wire', ''],
    ['TB_OUT_REG_COUNTER', 4, 'output wire', '']
]

  # output wire [13:0] TB_OUT_INTERP_0_0,
  # output wire [13:0] TB_OUT_INTERP_0_1,
  # output wire [13:0] TB_OUT_INTERP_0_2,
  # output wire [13:0] TB_OUT_INTERP_0_3,
  # output wire [13:0] TB_OUT_INTERP_0_4,
  # output wire [13:0] TB_OUT_INTERP_0_5,
  # output wire [13:0] TB_OUT_INTERP_0_6,
  # output wire [13:0] TB_OUT_INTERP_0_7,
  # output wire [13:0] TB_OUT_INTERP_0_8,
  # output wire [13:0] TB_OUT_INTERP_0_9,
  # output wire [13:0] TB_OUT_INTERP_0_10,
  # output wire [13:0] TB_OUT_INTERP_0_11,
  # output wire [13:0] TB_OUT_INTERP_0_12,
  # output wire [13:0] TB_OUT_INTERP_0_13,
  # output wire [13:0] TB_OUT_INTERP_0_14,

# Intermediary signals 
# Template: [signal name, signal type, signal width, description]
# Obs: signal type must be in accordance with the component's signal type
INT_SIGNALS = [
    ['OUT_REG_INPUT_LINE', 'wire', PIXEL_WIDTH*INPUT_LINE_SIZE, ''],
    ['EXT_OUT_REG_INPUT_LINE_0', 'wire signed', OUT_1ST_INTERP_WIDTH, ''],
    ['EXT_OUT_REG_INPUT_LINE_1', 'wire signed', OUT_1ST_INTERP_WIDTH, ''],
    ['EXT_OUT_REG_INPUT_LINE_2', 'wire signed', OUT_1ST_INTERP_WIDTH, ''],
    ['EXT_OUT_REG_INPUT_LINE_3', 'wire signed', OUT_1ST_INTERP_WIDTH, ''],
    ['EXT_OUT_REG_INPUT_LINE_4', 'wire signed', OUT_1ST_INTERP_WIDTH, ''],
    ['EXT_OUT_REG_INPUT_LINE_5', 'wire signed', OUT_1ST_INTERP_WIDTH, ''],
    ['EXT_OUT_REG_INPUT_LINE_6', 'wire signed', OUT_1ST_INTERP_WIDTH, ''],
    ['EXT_OUT_REG_INPUT_LINE_7', 'wire signed', OUT_1ST_INTERP_WIDTH, ''],
    ['EXT_OUT_REG_INPUT_LINE_8', 'wire signed', OUT_1ST_INTERP_WIDTH, ''],

    ['EXT_OUT_REG_INPUT_LINE_2_14', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['EXT_OUT_REG_INPUT_LINE_3_14', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['EXT_OUT_REG_INPUT_LINE_4_14', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['EXT_OUT_REG_INPUT_LINE_5_14', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],

    ['OUT_REG_COUNTER', 'wire', 4, ''],
    ['OUT_BUFFER_0', 'wire signed', OUT_1ST_INTERP_WIDTH*INPUT_LINE_SIZE, ''],
    ['OUT_BUFFER_1', 'wire signed', OUT_1ST_INTERP_WIDTH*INPUT_LINE_SIZE, ''],
    ['OUT_BUFFER_2', 'wire signed', OUT_1ST_INTERP_WIDTH*INPUT_LINE_SIZE, ''],
    ['OUT_BUFFER_3', 'wire signed', OUT_1ST_INTERP_WIDTH*INPUT_LINE_SIZE, ''],

    ['OUT_MUX_SEL_BUF_IN_0', 'wire', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_MUX_SEL_BUF_IN_1', 'wire', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_MUX_SEL_BUF_IN_2', 'wire', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_MUX_SEL_BUF_IN_3', 'wire', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_MUX_SEL_BUFFER_COLUMN', 'wire', OUT_1ST_INTERP_WIDTH*INPUT_LINE_SIZE, ''],
    ['OUT_MUX_SEL_INTERP_LINE_IN', 'wire', OUT_1ST_INTERP_WIDTH*INPUT_LINE_SIZE, ''],
    ['OUT_MUX_SEL_DIMENSION', 'wire', int(math.log2((FILTERS+1))), ''],
    ['OUT_MUX_SEL_FILTER_0', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_MUX_SEL_FILTER_1', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_MUX_SEL_FILTER_2', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_MUX_SEL_FILTER_3', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],

    ['OUT_ADD_COUNTER', 'wire', 4, ''],
    ['MSB_REG_INPUT_LINE', 'wire', (OUT_1ST_INTERP_WIDTH-PIXEL_WIDTH)*INPUT_LINE_SIZE, ''],
    ['EXT_OUT_REG_INPUT_LINE', 'wire', OUT_1ST_INTERP_WIDTH*INPUT_LINE_SIZE, '\t\t// Sign extension for OUT_REG_INPUT_LINE (72 -> 99 bits)', ''],

    ['OUT_INTERP_0_0', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_0_1', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_0_2', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_0_3', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_0_4', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_0_5', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_0_6', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_0_7', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_0_8', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_0_9', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_0_10', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_0_11', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_0_12', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_0_13', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_0_14', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],

    ['OUT_INTERP_1_0', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_1_1', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_1_2', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_1_3', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_1_4', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_1_5', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_1_6', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_1_7', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_1_8', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_1_9', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_1_10', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_1_11', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_1_12', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_1_13', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_1_14', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],

    ['OUT_INTERP_2_0', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_2_1', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_2_2', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_2_3', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_2_4', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_2_5', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_2_6', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_2_7', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_2_8', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_2_9', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_2_10', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_2_11', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_2_12', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_2_13', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_2_14', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],

    ['OUT_INTERP_3_0', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_3_1', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_3_2', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_3_3', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_3_4', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_3_5', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_3_6', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_3_7', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_3_8', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_3_9', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_3_10', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_3_11', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_3_12', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_3_13', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_3_14', 'wire signed', OUT_2ND_INTERP_WIDTH, '']
]


# Components informations 
# [component name, component type, [inputs names], [outputs], description] 
# Template: ['', '', [], , [], , ''],
INT_COMPONENTS = [
    ['REG_INPUT_LINE', 'reg_input_line', ['CLK', 'RST_ASYNC_INTERP', 'WRITE_REG_INPUT_LINE', 'INTEGER_SAMPLES', '', '', '', '', '', '', '', '', '', '', '', ''],
     ['OUT_REG_INPUT_LINE', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], '\t\t\t// Stores the external input samples'],

    ['REG_COUNTER', 'reg_counter', ['CLK', 'RST_ASYNC_REG_COUNTER', 'WRITE_REG_COUNTER', 'OUT_ADD_COUNTER', '', '', '', '', '', '', '', '', '', '', '', ''],
     ['OUT_REG_COUNTER', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], '\t\t\t// Controls the counter increment that controls the loops for FSM'],

    ['TRANSPOSED_BUFFER', 'transposed_buffer', 
     ['CLK', 'RST_ASYNC_INTERP', 'WRITE_BUFFER', 
      'OUT_MUX_SEL_BUF_IN_0[10:0]', 'OUT_MUX_SEL_BUF_IN_1[10:0]', 'OUT_MUX_SEL_BUF_IN_2[10:0]', 'OUT_MUX_SEL_BUF_IN_3[10:0]', '', '', '', '', '', '', '', '', ''], 
      ['OUT_BUFFER_0', 'OUT_BUFFER_1', 'OUT_BUFFER_2', 'OUT_BUFFER_3', '', '', '', '', '', '', '', '', '', '', ''], ''],

    ['REG_INT_OUT_0', 'reg_int_out', ['CLK', 'RST_ASYNC_INTERP', 'WRITE_REG_INT_OUT', 'OUT_MUX_SEL_BUF_IN_0', '', '', '', '', '', '', '', '', '', '', '', ''],
     ['INTERP_OUT_0', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], '\t\t\t// Output from interpolator 0'],

    ['REG_INT_OUT_1', 'reg_int_out', ['CLK', 'RST_ASYNC_INTERP', 'WRITE_REG_INT_OUT', 'OUT_MUX_SEL_BUF_IN_1', '', '', '', '', '', '', '', '', '', '', '', ''],
     ['INTERP_OUT_1', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], '\t\t\t// Output from interpolator 1'],

    ['REG_INT_OUT_2', 'reg_int_out', ['CLK', 'RST_ASYNC_INTERP', 'WRITE_REG_INT_OUT', 'OUT_MUX_SEL_BUF_IN_2', '', '', '', '', '', '', '', '', '', '', '', ''],
     ['INTERP_OUT_2', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], '\t\t\t// Output from interpolator 2'],

    ['REG_INT_OUT_3', 'reg_int_out', ['CLK', 'RST_ASYNC_INTERP', 'WRITE_REG_INT_OUT', 'OUT_MUX_SEL_BUF_IN_3', '', '', '', '', '', '', '', '', '', '', '', ''],
     ['INTERP_OUT_3', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], '\t\t\t// Output from interpolator 3'],

    ['ADD_COUNTER', 'add_counter', ["4'b1", 'OUT_REG_COUNTER', '', '', '', '', '', '', '', '', '', '', '', '', ''],
     ['OUT_ADD_COUNTER', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ''],

    ['MUX_SEL_BUF_IN_0', 'mux_sel_buffer_in', ['OUT_MUX_SEL_FILTER_0', 'EXT_OUT_REG_INPUT_LINE_2_14', 'SEL_BUFFER_IN', '', '', '', '', '', '', '', '', '', '', '', '', ''], 
     ['OUT_MUX_SEL_BUF_IN_0', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ''],

    ['MUX_SEL_BUF_IN_1', 'mux_sel_buffer_in', ['OUT_MUX_SEL_FILTER_1', 'EXT_OUT_REG_INPUT_LINE_3_14', 'SEL_BUFFER_IN', '', '', '', '', '', '', '', '', '', '', '', '', ''], 
     ['OUT_MUX_SEL_BUF_IN_1', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ''],

    ['MUX_SEL_BUF_IN_2', 'mux_sel_buffer_in', ['OUT_MUX_SEL_FILTER_2', 'EXT_OUT_REG_INPUT_LINE_4_14', 'SEL_BUFFER_IN', '', '', '', '', '', '', '', '', '', '', '', '', ''], 
     ['OUT_MUX_SEL_BUF_IN_2', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ''],

    ['MUX_SEL_BUF_IN_3', 'mux_sel_buffer_in', ['OUT_MUX_SEL_FILTER_3', 'EXT_OUT_REG_INPUT_LINE_5_14', 'SEL_BUFFER_IN', '', '', '', '', '', '', '', '', '', '', '', '', ''], 
     ['OUT_MUX_SEL_BUF_IN_3', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ''],

    ['MUX_SEL_BUFFER_COLUMN', 'mux_sel_buffer_column', ['OUT_BUFFER_0', 'OUT_BUFFER_1', 'OUT_BUFFER_2', 'OUT_BUFFER_3', 'OUT_REG_COUNTER[2:0]', '', '', '', '', '', '', '', '', '', '', ''],
     ['OUT_MUX_SEL_BUFFER_COLUMN', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ''],

    ['MUX_SEL_INTERP_LINE_IN', 'mux_sel_interp_line_in', ['OUT_MUX_SEL_BUFFER_COLUMN', 'EXT_OUT_REG_INPUT_LINE', 'SEL_INTERP_IN', '', '', '', '', '', '', '', '', '', '', '', '', ''],
     ['OUT_MUX_SEL_INTERP_LINE_IN', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ''],

    ['MUX_SEL_DIMENSION', 'mux_sel_dimension', ['FRAC_MV_Y', 'FRAC_MV_X', 'SEL_DIMENSION', '', '', '', '', '', '', '', '', '', '', '', '', ''], 
     ['OUT_MUX_SEL_DIMENSION', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], '\t\t// The output selects the filter'],

    ['MUX_SEL_FILTER_0', 'mux_sel_filter', ['OUT_INTERP_0_0', 'OUT_INTERP_0_1', 'OUT_INTERP_0_2', 'OUT_INTERP_0_3', 'OUT_INTERP_0_4', 
                                            'OUT_INTERP_0_5', 'OUT_INTERP_0_6', 'OUT_INTERP_0_7', 'OUT_INTERP_0_8', 'OUT_INTERP_0_9', 
                                            'OUT_INTERP_0_10', 'OUT_INTERP_0_11', 'OUT_INTERP_0_12', 'OUT_INTERP_0_13', 'OUT_INTERP_0_14',
                                            'OUT_MUX_SEL_DIMENSION'],
     ['OUT_MUX_SEL_FILTER_0', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ''],

    ['MUX_SEL_FILTER_1', 'mux_sel_filter', ['OUT_INTERP_1_0', 'OUT_INTERP_1_1', 'OUT_INTERP_1_2', 'OUT_INTERP_1_3', 'OUT_INTERP_1_4', 
                                            'OUT_INTERP_1_5', 'OUT_INTERP_1_6', 'OUT_INTERP_1_7', 'OUT_INTERP_1_8', 'OUT_INTERP_1_9', 
                                            'OUT_INTERP_1_10', 'OUT_INTERP_1_11', 'OUT_INTERP_1_12', 'OUT_INTERP_1_13', 'OUT_INTERP_1_14',
                                            'OUT_MUX_SEL_DIMENSION'],
     ['OUT_MUX_SEL_FILTER_1', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ''],

    ['MUX_SEL_FILTER_2', 'mux_sel_filter', ['OUT_INTERP_2_0', 'OUT_INTERP_2_1', 'OUT_INTERP_2_2', 'OUT_INTERP_2_3', 'OUT_INTERP_2_4', 
                                            'OUT_INTERP_2_5', 'OUT_INTERP_2_6', 'OUT_INTERP_2_7', 'OUT_INTERP_2_8', 'OUT_INTERP_2_9', 
                                            'OUT_INTERP_2_10', 'OUT_INTERP_2_11', 'OUT_INTERP_2_12', 'OUT_INTERP_2_13', 'OUT_INTERP_2_14',
                                            'OUT_MUX_SEL_DIMENSION'],
     ['OUT_MUX_SEL_FILTER_2', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ''],

    ['MUX_SEL_FILTER_3', 'mux_sel_filter', ['OUT_INTERP_3_0', 'OUT_INTERP_3_1', 'OUT_INTERP_3_2', 'OUT_INTERP_3_3', 'OUT_INTERP_3_4', 
                                            'OUT_INTERP_3_5', 'OUT_INTERP_3_6', 'OUT_INTERP_3_7', 'OUT_INTERP_3_8', 'OUT_INTERP_3_9', 
                                            'OUT_INTERP_3_10', 'OUT_INTERP_3_11', 'OUT_INTERP_3_12', 'OUT_INTERP_3_13', 'OUT_INTERP_3_14',
                                            'OUT_MUX_SEL_DIMENSION'],
     ['OUT_MUX_SEL_FILTER_3', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ''],

    ['INTERP_0', 'all_filters_11', ['OUT_MUX_SEL_INTERP_LINE_IN[98:33]', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
      ['OUT_INTERP_0_0', 'OUT_INTERP_0_1', 'OUT_INTERP_0_2', 'OUT_INTERP_0_3', 'OUT_INTERP_0_4', 
       'OUT_INTERP_0_5', 'OUT_INTERP_0_6', 'OUT_INTERP_0_7', 'OUT_INTERP_0_8', 'OUT_INTERP_0_9', 
       'OUT_INTERP_0_10', 'OUT_INTERP_0_11', 'OUT_INTERP_0_12', 'OUT_INTERP_0_13', 'OUT_INTERP_0_14'], ''],

    ['INTERP_1', 'all_filters_11', ['OUT_MUX_SEL_INTERP_LINE_IN[87:22]', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
     ['OUT_INTERP_1_0', 'OUT_INTERP_1_1', 'OUT_INTERP_1_2', 'OUT_INTERP_1_3', 'OUT_INTERP_1_4', 
      'OUT_INTERP_1_5', 'OUT_INTERP_1_6', 'OUT_INTERP_1_7', 'OUT_INTERP_1_8', 'OUT_INTERP_1_9', 
      'OUT_INTERP_1_10', 'OUT_INTERP_1_11', 'OUT_INTERP_1_12', 'OUT_INTERP_1_13', 'OUT_INTERP_1_14'], ''],

    ['INTERP_2', 'all_filters_11', ['OUT_MUX_SEL_INTERP_LINE_IN[76:11]', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
     ['OUT_INTERP_2_0', 'OUT_INTERP_2_1', 'OUT_INTERP_2_2', 'OUT_INTERP_2_3', 'OUT_INTERP_2_4', 
      'OUT_INTERP_2_5', 'OUT_INTERP_2_6', 'OUT_INTERP_2_7', 'OUT_INTERP_2_8', 'OUT_INTERP_2_9', 
      'OUT_INTERP_2_10', 'OUT_INTERP_2_11', 'OUT_INTERP_2_12', 'OUT_INTERP_2_13', 'OUT_INTERP_2_14'], ''],

    ['INTERP_3', 'all_filters_11', ['OUT_MUX_SEL_INTERP_LINE_IN[65:0]', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
     ['OUT_INTERP_3_0', 'OUT_INTERP_3_1', 'OUT_INTERP_3_2', 'OUT_INTERP_3_3', 'OUT_INTERP_3_4', 
      'OUT_INTERP_3_5', 'OUT_INTERP_3_6', 'OUT_INTERP_3_7', 'OUT_INTERP_3_8', 'OUT_INTERP_3_9', 
      'OUT_INTERP_3_10', 'OUT_INTERP_3_11', 'OUT_INTERP_3_12', 'OUT_INTERP_3_13', 'OUT_INTERP_3_14'], '']
]
 

In [75]:
####################################################
# MV GENERATOR DATAPATH CALCULATIONS
####################################################

################## Inputs declaration ##################
IO_DECLARATION = []
for i in range(len(INT_CTRL_SIGNALS)):
    if INT_CTRL_SIGNALS[i][1] == 1:
        if i != len(INT_CTRL_SIGNALS)-1:
            IO_DECLARATION.append('  '+INT_CTRL_SIGNALS[i][2]+' '+INT_CTRL_SIGNALS[i][0]+','+INT_CTRL_SIGNALS[i][3])
        else:
            IO_DECLARATION.append('  '+INT_CTRL_SIGNALS[i][2]+' '+INT_CTRL_SIGNALS[i][0]+INT_CTRL_SIGNALS[i][3])
    else:
        if i != len(INT_CTRL_SIGNALS)-1:
            IO_DECLARATION.append('  '+INT_CTRL_SIGNALS[i][2]+' ['+str(INT_CTRL_SIGNALS[i][1]-1)+':0] '+INT_CTRL_SIGNALS[i][0]+','+INT_CTRL_SIGNALS[i][3])
        else:
            IO_DECLARATION.append('  '+INT_CTRL_SIGNALS[i][2]+' ['+str(INT_CTRL_SIGNALS[i][1]-1)+':0] '+INT_CTRL_SIGNALS[i][0]+INT_CTRL_SIGNALS[i][3])    
IO_DECLARATION_CONCAT = '\n'.join(IO_DECLARATION)


################## Signals declaration ##################
SIG_DECLARATION = []
for i in range(len(INT_SIGNALS)):
    SIG_DECLARATION.append('  '+INT_SIGNALS[i][1]+' ['+str(INT_SIGNALS[i][2]-1)+':0] '+INT_SIGNALS[i][0]+';'+INT_SIGNALS[i][3])
SIG_DECLARATION_CONCAT = '\n'.join(SIG_DECLARATION)


################## Components declarations ##################
COMPONENTS_DECLARATION = []
for i in range(len(INT_COMPONENTS)):
    if (INT_COMPONENTS[i][0][:3] == 'REG'):
        COMPONENTS_DECLARATION.append('  '+INT_COMPONENTS[i][1]+' '+INT_COMPONENTS[i][0]+' (\t\t'+INT_COMPONENTS[i][4])
        COMPONENTS_DECLARATION.append('\t.CLK ('+INT_COMPONENTS[i][2][0]+'),')
        COMPONENTS_DECLARATION.append('\t.RST_ASYNC_N ('+INT_COMPONENTS[i][2][1]+'),')
        COMPONENTS_DECLARATION.append('\t.WRITE_EN ('+INT_COMPONENTS[i][2][2]+'),')
        COMPONENTS_DECLARATION.append('\t.DATA_IN ('+INT_COMPONENTS[i][2][3]+'),')
        COMPONENTS_DECLARATION.append('\t.DATA_OUT ('+INT_COMPONENTS[i][3][0]+'));\n')

    elif (INT_COMPONENTS[i][0][:3] == 'ADD'):
        COMPONENTS_DECLARATION.append('  '+INT_COMPONENTS[i][1]+' '+INT_COMPONENTS[i][0]+' (\t\t'+INT_COMPONENTS[i][4])
        COMPONENTS_DECLARATION.append('\t.A ('+INT_COMPONENTS[i][2][0]+'),')
        COMPONENTS_DECLARATION.append('\t.B ('+INT_COMPONENTS[i][2][1]+'),')
        COMPONENTS_DECLARATION.append('\t.C ('+INT_COMPONENTS[i][3][0]+'));\n')

    elif (INT_COMPONENTS[i][0] == 'MUX_SEL_BUFFER_COLUMN'):
        COMPONENTS_DECLARATION.append('  '+INT_COMPONENTS[i][1]+' '+INT_COMPONENTS[i][0]+' (\t\t'+INT_COMPONENTS[i][4])
        COMPONENTS_DECLARATION.append('\t.DATA_IN_2 ('+INT_COMPONENTS[i][2][0]+'),')
        COMPONENTS_DECLARATION.append('\t.DATA_IN_3 ('+INT_COMPONENTS[i][2][1]+'),')
        COMPONENTS_DECLARATION.append('\t.DATA_IN_4 ('+INT_COMPONENTS[i][2][2]+'),')
        COMPONENTS_DECLARATION.append('\t.DATA_IN_5 ('+INT_COMPONENTS[i][2][3]+'),')
        COMPONENTS_DECLARATION.append('\t.SELECT ('+INT_COMPONENTS[i][2][4]+'),')
        COMPONENTS_DECLARATION.append('\t.DATA_OUT ('+INT_COMPONENTS[i][3][0]+'));\n')

    elif (INT_COMPONENTS[i][0] == 'MUX_SEL_BUFFER_COLUMN'):
        COMPONENTS_DECLARATION.append('  '+INT_COMPONENTS[i][1]+' '+INT_COMPONENTS[i][0]+' (\t\t'+INT_COMPONENTS[i][4])
        COMPONENTS_DECLARATION.append('\t.DATA_IN_1 ('+INT_COMPONENTS[i][2][0]+'),')
        COMPONENTS_DECLARATION.append('\t.DATA_IN_2 ('+INT_COMPONENTS[i][2][1]+'),')
        COMPONENTS_DECLARATION.append('\t.DATA_IN_3 ('+INT_COMPONENTS[i][2][2]+'),')
        COMPONENTS_DECLARATION.append('\t.DATA_IN_4 ('+INT_COMPONENTS[i][2][3]+'),')
        COMPONENTS_DECLARATION.append('\t.SELECT ('+INT_COMPONENTS[i][2][4]+'),')
        COMPONENTS_DECLARATION.append('\t.DATA_OUT ('+INT_COMPONENTS[i][3][0]+'));\n')

    elif (INT_COMPONENTS[i][0][:14] == 'MUX_SEL_FILTER'):
        COMPONENTS_DECLARATION.append('  '+INT_COMPONENTS[i][1]+' '+INT_COMPONENTS[i][0]+' (\t\t'+INT_COMPONENTS[i][4])
        COMPONENTS_DECLARATION.append('\t.DATA_IN_1 ('+INT_COMPONENTS[i][2][0]+'),')
        COMPONENTS_DECLARATION.append('\t.DATA_IN_2 ('+INT_COMPONENTS[i][2][1]+'),')
        COMPONENTS_DECLARATION.append('\t.DATA_IN_3 ('+INT_COMPONENTS[i][2][2]+'),')
        COMPONENTS_DECLARATION.append('\t.DATA_IN_4 ('+INT_COMPONENTS[i][2][3]+'),')
        COMPONENTS_DECLARATION.append('\t.DATA_IN_5 ('+INT_COMPONENTS[i][2][4]+'),')
        COMPONENTS_DECLARATION.append('\t.DATA_IN_6 ('+INT_COMPONENTS[i][2][5]+'),')
        COMPONENTS_DECLARATION.append('\t.DATA_IN_7 ('+INT_COMPONENTS[i][2][6]+'),')
        COMPONENTS_DECLARATION.append('\t.DATA_IN_8 ('+INT_COMPONENTS[i][2][7]+'),')
        COMPONENTS_DECLARATION.append('\t.DATA_IN_9 ('+INT_COMPONENTS[i][2][8]+'),')
        COMPONENTS_DECLARATION.append('\t.DATA_IN_10 ('+INT_COMPONENTS[i][2][9]+'),')
        COMPONENTS_DECLARATION.append('\t.DATA_IN_11 ('+INT_COMPONENTS[i][2][10]+'),')
        COMPONENTS_DECLARATION.append('\t.DATA_IN_12 ('+INT_COMPONENTS[i][2][11]+'),')
        COMPONENTS_DECLARATION.append('\t.DATA_IN_13 ('+INT_COMPONENTS[i][2][12]+'),')
        COMPONENTS_DECLARATION.append('\t.DATA_IN_14 ('+INT_COMPONENTS[i][2][13]+'),')
        COMPONENTS_DECLARATION.append('\t.DATA_IN_15 ('+INT_COMPONENTS[i][2][14]+'),')
        COMPONENTS_DECLARATION.append('\t.SELECT ('+INT_COMPONENTS[i][2][15]+'),')
        COMPONENTS_DECLARATION.append('\t.DATA_OUT ('+INT_COMPONENTS[i][3][0]+'));\n')

    elif (INT_COMPONENTS[i][0][:3] == 'MUX'):
        COMPONENTS_DECLARATION.append('  '+INT_COMPONENTS[i][1]+' '+INT_COMPONENTS[i][0]+' (\t\t'+INT_COMPONENTS[i][4])
        COMPONENTS_DECLARATION.append('\t.DATA_IN_0 ('+INT_COMPONENTS[i][2][0]+'),')
        COMPONENTS_DECLARATION.append('\t.DATA_IN_1 ('+INT_COMPONENTS[i][2][1]+'),')
        COMPONENTS_DECLARATION.append('\t.SELECT ('+INT_COMPONENTS[i][2][2]+'),')        
        COMPONENTS_DECLARATION.append('\t.DATA_OUT ('+INT_COMPONENTS[i][3][0]+'));\n')   

    elif (INT_COMPONENTS[i][0][:6] == 'INTERP'):
        COMPONENTS_DECLARATION.append('  '+INT_COMPONENTS[i][1]+' '+INT_COMPONENTS[i][0]+' (\t\t'+INT_COMPONENTS[i][4])
        COMPONENTS_DECLARATION.append('\t.X ('+INT_COMPONENTS[i][2][0]+'),')
        COMPONENTS_DECLARATION.append('\t.Y1 ('+INT_COMPONENTS[i][3][0]+'),')
        COMPONENTS_DECLARATION.append('\t.Y2 ('+INT_COMPONENTS[i][3][1]+'),')
        COMPONENTS_DECLARATION.append('\t.Y3 ('+INT_COMPONENTS[i][3][2]+'),')
        COMPONENTS_DECLARATION.append('\t.Y4 ('+INT_COMPONENTS[i][3][3]+'),')
        COMPONENTS_DECLARATION.append('\t.Y5 ('+INT_COMPONENTS[i][3][4]+'),')
        COMPONENTS_DECLARATION.append('\t.Y6 ('+INT_COMPONENTS[i][3][5]+'),')
        COMPONENTS_DECLARATION.append('\t.Y7 ('+INT_COMPONENTS[i][3][6]+'),')
        COMPONENTS_DECLARATION.append('\t.Y8 ('+INT_COMPONENTS[i][3][7]+'),')
        COMPONENTS_DECLARATION.append('\t.Y9 ('+INT_COMPONENTS[i][3][8]+'),')
        COMPONENTS_DECLARATION.append('\t.Y10 ('+INT_COMPONENTS[i][3][9]+'),')
        COMPONENTS_DECLARATION.append('\t.Y11 ('+INT_COMPONENTS[i][3][10]+'),')
        COMPONENTS_DECLARATION.append('\t.Y12 ('+INT_COMPONENTS[i][3][11]+'),')
        COMPONENTS_DECLARATION.append('\t.Y13 ('+INT_COMPONENTS[i][3][12]+'),')
        COMPONENTS_DECLARATION.append('\t.Y14 ('+INT_COMPONENTS[i][3][13]+'),')
        COMPONENTS_DECLARATION.append('\t.Y15 ('+INT_COMPONENTS[i][3][14]+'));\n')

    elif (INT_COMPONENTS[i][0][:3] == 'TRA'):
        COMPONENTS_DECLARATION.append('  '+INT_COMPONENTS[i][1]+' '+INT_COMPONENTS[i][0]+' (\t\t'+INT_COMPONENTS[i][4])
        COMPONENTS_DECLARATION.append('\t.CLK ('+INT_COMPONENTS[i][2][0]+'),')
        COMPONENTS_DECLARATION.append('\t.RST_ASYNC_N ('+INT_COMPONENTS[i][2][1]+'),')
        COMPONENTS_DECLARATION.append('\t.WRITE_EN ('+INT_COMPONENTS[i][2][2]+'),')
        COMPONENTS_DECLARATION.append('\t.INPUT_0 ('+INT_COMPONENTS[i][2][3]+'),')
        COMPONENTS_DECLARATION.append('\t.INPUT_1 ('+INT_COMPONENTS[i][2][4]+'),')
        COMPONENTS_DECLARATION.append('\t.INPUT_2 ('+INT_COMPONENTS[i][2][5]+'),')
        COMPONENTS_DECLARATION.append('\t.INPUT_3 ('+INT_COMPONENTS[i][2][6]+'),')
        COMPONENTS_DECLARATION.append('\t.OUTPUT_0 ('+INT_COMPONENTS[i][3][0]+'),')
        COMPONENTS_DECLARATION.append('\t.OUTPUT_1 ('+INT_COMPONENTS[i][3][1]+'),')
        COMPONENTS_DECLARATION.append('\t.OUTPUT_2 ('+INT_COMPONENTS[i][3][2]+'),')
        COMPONENTS_DECLARATION.append('\t.OUTPUT_3 ('+INT_COMPONENTS[i][3][3]+'));\n')
        
COMPONENTS_DECLARATION_CONCAT = '\n'.join(COMPONENTS_DECLARATION)


################## Combinational logic ##################
COMBINATIONAL_LOGIC = []
COMBINATIONAL_LOGIC.append('// Size matching:')
EXT_11_CONCAT = []
for g in range(INPUT_LINE_SIZE):   
    EXT_LINE = []
    EXT_LINE.append('  assign EXT_OUT_REG_INPUT_LINE_'+str(g)+' = {') 
    EXT_11_CONCAT.append('EXT_OUT_REG_INPUT_LINE_'+str(g))
    for r in range(OUT_1ST_INTERP_WIDTH-INPUT_LINE_SIZE+1):
        EXT_LINE.append('OUT_REG_INPUT_LINE['+str(INPUT_LINE_SIZE*PIXEL_WIDTH-1-g*PIXEL_WIDTH)+'], ')
    EXT_LINE.append('OUT_REG_INPUT_LINE['+str(INPUT_LINE_SIZE*PIXEL_WIDTH-1-g*PIXEL_WIDTH)+':'+str(INPUT_LINE_SIZE*PIXEL_WIDTH-g*PIXEL_WIDTH-PIXEL_WIDTH)+']};')
    COMBINATIONAL_LOGIC.append(''.join(EXT_LINE))
COMBINATIONAL_LOGIC.append('  assign EXT_OUT_REG_INPUT_LINE = {'+(', '.join(EXT_11_CONCAT))+'};\n')

for x in range(2,6):
    EXT_14_BITS = []
    EXT_14_BITS.append('  assign EXT_OUT_REG_INPUT_LINE_'+str(x)+'_14 = {')
    for y in range(OUT_2ND_INTERP_WIDTH-INPUT_LINE_SIZE+1):
        EXT_14_BITS.append('OUT_REG_INPUT_LINE['+str(INPUT_LINE_SIZE*PIXEL_WIDTH-1-x*PIXEL_WIDTH)+'], ')
    EXT_14_BITS.append('OUT_REG_INPUT_LINE['+str(INPUT_LINE_SIZE*PIXEL_WIDTH-1-x*PIXEL_WIDTH)+':'+str(INPUT_LINE_SIZE*PIXEL_WIDTH-x*PIXEL_WIDTH-PIXEL_WIDTH)+']};')
    COMBINATIONAL_LOGIC.append(''.join(EXT_14_BITS))

COMBINATIONAL_LOGIC.append('\n// Output loops for FSM ')
COMBINATIONAL_LOGIC.append("  assign LOOP_3 = OUT_REG_COUNTER[3] ^ 1'b0 | OUT_REG_COUNTER[2] ^ 1'b0 | OUT_REG_COUNTER[1] ^ 1'b1 | OUT_REG_COUNTER[0] ^ 1'b1;")
COMBINATIONAL_LOGIC.append("  assign LOOP_4 = OUT_REG_COUNTER[3] ^ 1'b0 | OUT_REG_COUNTER[2] ^ 1'b1 | OUT_REG_COUNTER[1] ^ 1'b0 | OUT_REG_COUNTER[0] ^ 1'b0;")
COMBINATIONAL_LOGIC.append("  assign LOOP_9 = OUT_REG_COUNTER[3] ^ 1'b1 | OUT_REG_COUNTER[2] ^ 1'b0 | OUT_REG_COUNTER[1] ^ 1'b0 | OUT_REG_COUNTER[0] ^ 1'b1;")
COMBINATIONAL_LOGIC.append("  assign LOOP_13 = OUT_REG_COUNTER[3] ^ 1'b1 | OUT_REG_COUNTER[2] ^ 1'b1 | OUT_REG_COUNTER[1] ^ 1'b0 | OUT_REG_COUNTER[0] ^ 1'b1;")

###################### TESTBENCH ######################
COMBINATIONAL_LOGIC.append('\n// Outputs for TESTBENCH ')
COMBINATIONAL_LOGIC.append('  assign TB_OUT_REG_INPUT_LINE = OUT_REG_INPUT_LINE;')
COMBINATIONAL_LOGIC.append('  assign TB_OUT_MUX_SEL_FILTER_0 = OUT_MUX_SEL_FILTER_0;')
COMBINATIONAL_LOGIC.append('  assign TB_OUT_MUX_SEL_FILTER_1 = OUT_MUX_SEL_FILTER_1;')
COMBINATIONAL_LOGIC.append('  assign TB_OUT_MUX_SEL_FILTER_2 = OUT_MUX_SEL_FILTER_2;')
COMBINATIONAL_LOGIC.append('  assign TB_OUT_MUX_SEL_FILTER_3 = OUT_MUX_SEL_FILTER_3;')
COMBINATIONAL_LOGIC.append('  assign TB_BUF_COL_0 = OUT_BUFFER_0;')
COMBINATIONAL_LOGIC.append('  assign TB_BUF_COL_1 = OUT_BUFFER_1;')
COMBINATIONAL_LOGIC.append('  assign TB_BUF_COL_2 = OUT_BUFFER_2; ')
COMBINATIONAL_LOGIC.append('  assign TB_BUF_COL_3 = OUT_BUFFER_3;')
COMBINATIONAL_LOGIC.append('  assign TB_OUT_MUX_SEL_INTERP_LINE_IN = OUT_MUX_SEL_INTERP_LINE_IN;')
COMBINATIONAL_LOGIC.append('  assign TB_IN_ALL_FILTERS_0 = OUT_MUX_SEL_INTERP_LINE_IN[98:33];')
COMBINATIONAL_LOGIC.append('  assign TB_IN_ALL_FILTERS_1 = OUT_MUX_SEL_INTERP_LINE_IN[87:22];')
COMBINATIONAL_LOGIC.append('  assign TB_IN_ALL_FILTERS_2 = OUT_MUX_SEL_INTERP_LINE_IN[76:11];')
COMBINATIONAL_LOGIC.append('  assign TB_IN_ALL_FILTERS_3 = OUT_MUX_SEL_INTERP_LINE_IN[65:0];')
COMBINATIONAL_LOGIC.append('  assign TB_OUT_ADD_COUNTER = OUT_ADD_COUNTER;')
COMBINATIONAL_LOGIC.append('  assign TB_OUT_REG_COUNTER = OUT_REG_COUNTER;')

COMBINATIONAL_LOGIC_CONCAT = '\n'.join(COMBINATIONAL_LOGIC)


# print(IO_DECLARATION_CONCAT)
# print(SIG_DECLARATION_CONCAT)
# print(COMPONENTS_DECLARATION_CONCAT)
# print(COMBINATIONAL_LOGIC_CONCAT)

### Datapath text

In [76]:
#########################################
# Text writing: Interpolation datapath
#########################################
INT_DATAPATH_VERILOG = '''/*------------------------------------------------------------------------------
 * File: '''+INT_FILE_NAME+'''.v
 * Date generated: 13/03/2023
 * Date modified: '''+DATE_MODIF+'''
 * Author: Bruna Suemi Nagai
 * Description: '''+INT_DESCRIPTION+'''
 *------------------------------------------------------------------------------ */

module '''+INT_FILE_NAME+''' (
'''+IO_DECLARATION_CONCAT+'''
);


// ------------------------------------------
// Signals declarations
// ------------------------------------------
'''+SIG_DECLARATION_CONCAT+'''

// ------------------------------------------
// Modules instantiation
// ------------------------------------------
'''+COMPONENTS_DECLARATION_CONCAT+'''
    

// ------------------------------------------
// Combinational logic
// ------------------------------------------
'''+COMBINATIONAL_LOGIC_CONCAT+'''


endmodule // '''+INT_FILE_NAME+'''    
'''

# print(INT_DATAPATH_VERILOG)
with open('../verilog/datapaths/'+INT_FILE_NAME+'.v', 'w') as int_datapath_file:
    int_datapath_file.write(INT_DATAPATH_VERILOG)

